In [2]:
from dotenv import load_dotenv
load_dotenv()

import asyncio

from computer_use_demo.loop import sampling_loop, PROVIDER_TO_DEFAULT_MODEL_NAME, APIProvider, SYSTEM_PROMPT
import os

from playwright.async_api import async_playwright
from termcolor import colored

In [3]:
playwright = await async_playwright().start()
browser = await playwright.chromium.launch(headless=False)
page = await browser.new_page(viewport={"width": 1024, "height": 768})

In [4]:
messages = []

In [5]:
def prefixed_print(content, prefix=""):
    print(*[prefix + line for line in content.split("\n")], sep="\n")

def print_conversation():
    for message in messages:
            role = message.get("role")
            content = message.get("content")
            
            on_color = "on_green" if role == "user" else "on_red"
            color = "green" if role == "user" else "red"
            print(colored(f"[{role}]:", "black", on_color, ["bold"]))
            
            def print_content(content, prefix=""):
                if isinstance(content, str):
                    print(colored(content, color))
                    return
                
                for content_part in content:
                    if isinstance(content_part, str):
                        print(colored(content_part, color))
                        continue
                    c_type = content_part.get("type")
                    if c_type == "text":
                        prefixed_print(colored(content_part.get("text"), color), prefix)
                    elif c_type == "image":
                        prefixed_print(colored("Image", "yellow"), prefix)
                        print_content([content_part.get("source")], prefix + "  ")
                    elif c_type == "base64":
                        prefixed_print(colored(f"[{content_part.get("media_type")}]: <{content_part.get("data")[:10]}...>", "yellow"), prefix)
                    elif c_type == "tool_use":
                        prefixed_print(colored(f"Using tool {content_part.get("name")} with input {content_part.get("input")}", "blue"), prefix)
                    elif c_type == "tool_result":
                        prefixed_print(colored(f"Tool result:", "blue"), prefix)
                        print_content(content_part.get("content"), prefix + "  ")
            
            print_content(content)  
            print("\n")
            
def print_stuff(prefix, *args, **kwargs):
    # pass
    print(f"[{prefix}]", args, kwargs)
    
def noop(*_, **__):
    pass
            
async def converse():
    await sampling_loop(
            model=PROVIDER_TO_DEFAULT_MODEL_NAME[APIProvider.ANTHROPIC],
            api_key=os.environ.get("ANTHROPIC_API_KEY"),
            provider=APIProvider.ANTHROPIC,
            messages=messages,
            system_prompt_suffix="",
            output_callback=print_stuff,
            tool_output_callback=print_stuff,
            api_response_callback=noop,
            page=page,
        )
    await asyncio.sleep(0.1)

In [6]:
user_message = input()
if len(user_message) > 0:
    messages.append({
        "role": "user",
        "content": user_message
    })
    await converse()
    print_conversation()

[{'type': 'text', 'text': "I'll help you check Elon Musk's latest tweet on X (formerly Twitter). Let me navigate to his profile."}] () {}
[{'id': 'toolu_01TPEEchduP42Ga1ubk28USN', 'input': {'action': '{"type": "goto", "url": "https://twitter.com/elonmusk"}'}, 'name': 'playwright', 'type': 'tool_use'}] () {}
[ToolResult(output='Navigated to page https://twitter.com/elonmusk', error=None, base64_image=None, system=None)] ('toolu_01TPEEchduP42Ga1ubk28USN',) {}
[{'type': 'text', 'text': 'Let me take a screenshot to see his latest tweets.'}] () {}
[{'id': 'toolu_016ijvPVjb29uuSP7PMWJdQD', 'input': {'action': 'screenshot'}, 'name': 'computer', 'type': 'tool_use'}] () {}
[ToolResult(output='Taken screenshot', error='', base64_image='...', system=None)] ('toolu_016ijvPVjb29uuSP7PMWJdQD',) {}
[user]:
what is the latest tweet by elon musk


[assistant]:
I'll help you check Elon Musk's latest tweet on X (formerly Twitter). Let me navigate to his profile.
Using tool playwright with input {'action'

In [7]:
print_conversation()

[user]:
what is the latest tweet by elon musk


[assistant]:
I'll help you check Elon Musk's latest tweet on X (formerly Twitter). Let me navigate to his profile.
Using tool playwright with input {'action': '{"type": "goto", "url": "https://twitter.com/elonmusk"}'}


[user]:
Tool result:
  Navigated to page https://twitter.com/elonmusk


[assistant]:
Let me take a screenshot to see his latest tweets.
Using tool computer with input {'action': 'screenshot'}


[user]:
Tool result:
  Taken screenshot
  Image
    [image/png]: <iVBORw0KGg...>




In [8]:
await browser.close()
await playwright.stop()